<a href="https://colab.research.google.com/github/risa0122/FBA/blob/main/data_test_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Data Processing

In [ ]:
import pandas as pd

data = pd.read_csv('data_asset_choosed.csv')
data.head()
data.shape
data = data.bfill()


In [ ]:
data.head()

,Unnamed: 0,NII@NI225,IKS200,IKQ150,A132030,SPI@SPX
0,2010-10-01,NIKKEI 225 INDEX,코스피 200,코스닥 150,KODEX 골드선물(H),S&P 500
1,2010-10-01,9404,243,968,9995,1146
2,2010-10-04,9381,244,964,10065,1137
3,2010-10-05,9519,244,964,10065,1161
4,2010-10-06,9691,248,971,10285,1160


In [ ]:
asset_names = data.loc[0].tolist()
del asset_names[0]
print(len(asset_names))
print(asset_names)

5
['NIKKEI 225 INDEX', '코스피 200', '코스닥 150', 'KODEX 골드선물(H)', 'S&P 500']


In [ ]:
data = data.drop(0,0)
data.head()

,Unnamed: 0,NII@NI225,IKS200,IKQ150,A132030,SPI@SPX
1,2010-10-01,9404,243,968,9995,1146
2,2010-10-04,9381,244,964,10065,1137
3,2010-10-05,9519,244,964,10065,1161
4,2010-10-06,9691,248,971,10285,1160
5,2010-10-07,9685,247,967,10335,1158


In [ ]:
data.rename(columns={data.columns[0]:"Date"},inplace=True)
data.head()

,Date,NII@NI225,IKS200,IKQ150,A132030,SPI@SPX
1,2010-10-01,9404,243,968,9995,1146
2,2010-10-04,9381,244,964,10065,1137
3,2010-10-05,9519,244,964,10065,1161
4,2010-10-06,9691,248,971,10285,1160
5,2010-10-07,9685,247,967,10335,1158


In [ ]:
data.set_index('Date')

,NII@NI225,IKS200,IKQ150,A132030,SPI@SPX
Date,,,,,
2010-10-01,9404,243,968,9995,1146
2010-10-04,9381,244,964,10065,1137
2010-10-05,9519,244,964,10065,1161
2010-10-06,9691,248,971,10285,1160
2010-10-07,9685,247,967,10335,1158
...,...,...,...,...,...
2020-05-21,20552,264,1093,12300,2949
2020-05-22,20388,260,1080,12260,2955
2020-05-25,20742,263,1096,12240,2992


In [ ]:
asset_ID = []
for i in range(len(data.columns)) :
  asset_ID.append(data.columns[i])
del asset_ID[0]
print(asset_ID)
#asset_ID[0]= 'period'

['NII@NI225', 'IKS200', 'IKQ150', 'A132030', 'SPI@SPX']


#수익률 table

In [ ]:
## make data frame
columns = ['period_buy','period_sell']+ asset_ID
data_temp = pd.DataFrame(columns=[columns])
#data_temp.reindex(data_temp.columns.tolist()+[asset_ID])
data_temp.head()

,period_buy,period_sell,NII@NI225,IKS200,IKQ150,A132030,SPI@SPX


In [ ]:
#del asset_ID[0]

test_period = 200
for i in range(len(data)-test_period):
  for id in asset_ID:
    st = float(data.iloc[i].loc[id])
    end = float(data.iloc[i+test_period].loc[id])
    profit = ((end-st)/st)*100
    data_temp.loc[i,'period_buy'] = data.iloc[i,0]
    data_temp.loc[i,'period_sell'] = data.iloc[i+test_period,0]
    data_temp.iloc[i].loc[id] = profit


In [ ]:
data_temp.head()

,period_buy,period_sell,NII@NI225,IKS200,IKQ150,A132030,SPI@SPX
0,2010-10-01,2011-07-20,6.40153,15.2263,8.78099,21.5608,15.7068
1,2010-10-04,2011-07-21,6.70504,14.3443,8.92116,21.4108,18.2058
2,2010-10-05,2011-07-22,6.43975,15.9836,10.3734,20.5166,15.8484
3,2010-10-06,2011-07-25,3.70447,12.9032,9.78373,20.0292,15.2586
4,2010-10-07,2011-07-26,4.26433,14.17,11.4788,19.4969,15.0259


In [ ]:
print(asset_ID)

['A069500', 'A143850', 'A195930', 'A192090', 'A148070', 'A136340', 'A182490', 'A132030', 'A130680']


In [ ]:
data_temp.to_excel('return_252.xlsx') # saving data

#Backtest

In [ ]:
date = []
total_account_value = []
import math
asset = 10000000
columns = ['date','buy/sell']+asset_ID
data_r = pd.DataFrame(columns=[columns])
a= 0
i = 0
rebalance = 15 # 변경 가능
li_change = []
while i < len(data)-1.5*rebalance :
  buy_list = []
  buy_date = data.loc[data['Date'] == data_temp.loc[i,'period_sell'].item()].index.item()+1
  #print(buy_date)
  sell_date = buy_date + rebalance
  total = 0
  #####buy index0#####
  data_r.loc[a,'date'] = data.loc[buy_date,'Date']
  data_r.loc[a,'buy/sell'] = 'buy_ratio'
  for id in asset_ID:
      if float(data_temp.loc[i,id]) > 1.3 : ### 이자율 --> 매수 조건
        buy_list.append(id)
        #print(buy_list)
  for id in buy_list:
    total = data_temp.loc[i,id].item()+total
  for id in asset_ID:
    if id in buy_list:
      data_r.loc[a,id] = (data_temp.loc[i,id].item()/total)*100
    else :
      data_r.loc[a,id] = 0
  #####buy index1#####
  data_r.loc[a+1,'date'] = data.loc[buy_date,'Date']
  data_r.loc[a+1,'buy/sell'] = 'buy_stock_number'
  for id in asset_ID:
    if id in buy_list:
      data_r.loc[a+1,id] =math.floor(((data_temp.loc[i,id].item()/total)*asset)/float(data.loc[buy_date,id]))##1000만원 기준
    else :
      data_r.loc[a+1,id] = 0
    #####buy index2#####
  data_r.loc[a+2,'date'] = data.loc[buy_date,'Date']
  data_r.loc[a+2,'buy/sell'] = 'buy_asset_value'
  asset_total = 0
  for id in asset_ID:
    if id in buy_list:
      data_r.loc[a+2,id] = data_r.loc[a+1,id].item()*float(data.loc[buy_date,id])
      asset_total = data_r.loc[a+2,id].item() + asset_total
    else :
      data_r.loc[a+2,id] = 0
  data_r.loc[a+2,'total_asset_value'] = asset_total
  data_r.loc[a+2,'left'] = asset - asset_total
  data_r.loc[a+2,'total'] = asset_total + data_r.loc[a+2,'left'].item()
  ####sell index3#####
  data_r.loc[a+3,'date'] = data.loc[sell_date,'Date']
  data_r.loc[a+3,'buy/sell'] = 'sell_asset_value'
  asset_total =  0
  for id in asset_ID:
    if id in buy_list:
      data_r.loc[a+3,id] = data_r.loc[a+1,id].item()*float(data.loc[sell_date,id])
      asset_total = data_r.loc[a+3,id].item() + asset_total
    else :
      data_r.loc[a+3,id] = 0
  data_r.loc[a+3,'total_asset_value'] = asset_total
  data_r.loc[a+3,'total'] = data_r.loc[a+3,'total_asset_value'].item() + data_r.loc[a+2,'left'].item()
  asset = data_r.loc[a+3,'total'].item()
  #####sell index4#####
  data_r.loc[a+4,'date'] = data.loc[sell_date,'Date']
  data_r.loc[a+4,'buy/sell'] = 'profit'
  asset_total =  0
  for id in asset_ID:
    if id in buy_list:
      data_r.loc[a+4,id] = data_r.loc[a+3,id].item()-data_r.loc[a+2,id].item()
    else :
      data_r.loc[a+4,id] = 0
  data_r.loc[a+4,'total_asset_value'] = data_r.loc[a+3,'total_asset_value']-data_r.loc[a+2,'total_asset_value']
  data_r.loc[a+4,'total'] = round(((data_r.loc[a+3,'total'].item()-data_r.loc[a+2,'total'].item())/data_r.loc[a+2,'total'].item())*100,2)
  print(i,data_r.loc[a+1,'date'].item(),data_r.loc[a+4,'total'].item(),asset)
   
  li_change.append(data_r.loc[a+4,'total'].item())
  a = a + 5
  i = i + rebalance
data_r.head()

0 2011-10-05 6.17 10617355.0
15 2011-10-26 2.22 10852805.0
30 2011-11-16 -1.48 10692342.0
45 2011-12-07 -7.04 9939484.0
60 2011-12-28 4.53 10389384.0
75 2012-01-19 3.74 10777934.0
90 2012-02-13 -1.08 10661979.0
105 2012-03-06 -0.24 10635869.0
120 2012-03-27 -2.73 10345145.0
135 2012-04-18 -2.62 10073782.0
150 2012-05-10 -2.57 9814537.0
165 2012-06-01 1.3 9942102.0
180 2012-06-25 2.38 10178372.0
195 2012-07-16 2.95 10479052.0
210 2012-08-06 1.08 10591807.0
225 2012-08-28 3.55 10967884.0
240 2012-09-18 -3.1 10628118.0
255 2012-10-11 -1.07 10514508.0
270 2012-11-01 -1.17 10391752.0
285 2012-11-22 2.14 10614380.0
300 2012-12-13 3.27 10961568.0
315 2013-01-09 2.85 11273577.0
330 2013-01-30 1.07 11394282.0
345 2013-02-21 8.68 12383467.0
360 2013-03-15 1.21 12533604.0
375 2013-04-05 5.79 13259213.0
390 2013-04-26 8.34 14364527.0
405 2013-05-21 -10.01 12926518.0
420 2013-06-12 3.38 13362841.0
435 2013-07-03 4.63 13981481.0
450 2013-07-24 -2.98 13564858.0
465 2013-08-14 -0.95 13436546.0
480 201

KeyError: ignored

In [ ]:
print(date)


['2011-10-28', '2011-11-22', '2011-12-15', '2012-01-10', '2012-02-06', '2012-02-29', '2012-03-26', '2012-04-19', '2012-05-15', '2012-06-11', '2012-07-04', '2012-07-27', '2012-08-22', '2012-09-14', '2012-10-11', '2012-11-05', '2012-11-28', '2012-12-24', '2013-01-21', '2013-02-14', '2013-03-12', '2013-04-04', '2013-04-29', '2013-05-24', '2013-06-19', '2013-07-12', '2013-08-06', '2013-08-30', '2013-09-27', '2013-10-24', '2013-11-18', '2013-12-11', '2014-01-08', '2014-02-04', '2014-02-27', '2014-03-24', '2014-04-16', '2014-05-14', '2014-06-10', '2014-07-03', '2014-07-28', '2014-08-21', '2014-09-18', '2014-10-15', '2014-11-07', '2014-12-02', '2014-12-26', '2015-01-22', '2015-02-16', '2015-03-16', '2015-04-08', '2015-05-04', '2015-05-29', '2015-06-23', '2015-07-16', '2015-08-10', '2015-09-03', '2015-09-30', '2015-10-26', '2015-11-18', '2015-12-11', '2016-01-08', '2016-02-02', '2016-03-02', '2016-03-25', '2016-04-20', '2016-05-17', '2016-06-10', '2016-07-05', '2016-07-28', '2016-08-23', '2016

In [ ]:
print(total_account_value)

[10758580.0, 10304783.0, 9573698.0, 9901288.0, 10564458.0, 10868906.0, 10317481.0, 10070351.0, 9634229.0, 9937229.0, 10233398.0, 10270462.0, 10470532.0, 10815764.0, 10474143.0, 10294790.0, 10327933.0, 10743616.0, 11141769.0, 11480350.0, 12235923.0, 12451060.0, 13252609.0, 13801714.0, 12823141.0, 13743740.0, 13730511.0, 12833159.0, 13934371.0, 13831205.0, 14359429.0, 14547177.0, 15052866.0, 13535755.0, 14381242.0, 14140557.0, 14133997.0, 14280704.0, 14759716.0, 15051718.0, 15117664.0, 15124864.0, 15334738.0, 14284400.0, 15739066.0, 16243956.0, 16336542.0, 16258735.0, 16710795.0, 17273387.0, 17677608.0, 17596392.0, 18397609.0, 18862009.0, 19153861.0, 18911479.0, 16872423.0, 16353760.0, 17113052.0, 17405508.0, 16845319.0, 17352840.0, 17793730.0, 16686556.0, 16686556.0, 16818820.0, 17238580.0, 17063680.0, 18130570.0, 18091648.0, 18090959.0, 17811977.0, 17384093.0, 17560994.0, 16627984.0, 16707730.0, 16951120.0, 17293757.0, 17520185.0, 17342355.0, 17643921.0, 18304289.0, 18515993.0, 1873434

In [ ]:
data_r.to_excel('buysell_cal400_re13.xlsx') # saving data

In [ ]:
data_r.tail(50)

In [ ]:
#portfolio
## variance
import numpy as np
print(li_change)
print(np.var(li_change))




[7.59, -4.22, -7.09, 3.42, 6.7, 2.88, -5.07, -2.4, -4.33, 3.15, 2.98, 0.36, 1.95, 3.3, -3.16, -1.71, 0.32, 4.02, 3.71, 3.04, 6.58, 1.76, 6.44, 4.14, -7.09, 7.18, -0.1, -6.54, 8.58, -0.74, 3.82, 1.31, 3.48, -10.08, 6.25, -1.67, -0.05, 1.04, 3.35, 1.98, 0.44, 0.05, 1.39, -6.85, 10.18, 3.21, 0.57, -0.48, 2.78, 3.37, 2.34, -0.46, 4.55, 2.52, 1.55, -1.27, -10.78, -3.07, 4.64, 1.71, -3.22, 3.01, 2.54, -6.22, 0.0, 0.79, 2.5, -1.01, 6.25, -0.21, -0.0, -1.54, -2.4, 1.02, -5.31, 0.48, 1.46, 2.02, 1.31, -1.02, 1.74, 3.74, 1.16, 1.18, -1.5, -1.36, 3.32, 5.99, 5.24, -2.29, 11.81, -4.62, -0.33, -0.6, -1.45, -1.26, 0.36, -3.34, -4.23, 2.96, 4.14, -6.93, -7.37, -2.55, -3.84, 0.0, 0.0, 0.37, 3.78, -3.66, 0.5, 4.88, 2.24, 2.19, -1.02, 0.33, 0.28, -0.5, 1.93, 3.27, 0.41, -11.03, -5.8, 8.66, -0.41]
16.6725508096
